<a href="https://colab.research.google.com/github/ved1beta/car_auction/blob/main/titanic_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>